# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Seu código aqui
URL = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Município'

In [3]:
r = requests.get(URL)

In [4]:
data_json = r.json()

In [5]:
df = pd.DataFrame(data_json['items'])

In [6]:
df.head(3)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,7993,Município,Rio Claro,3304409,RJ,None,Deferido,17944.000691/2010-74,2011-06-01T03:00:00Z,Operação contratual interna,Caminho da Escola 2009 A,Instituição Financeira Nacional,Banco do Brasil S/A,Real,584500.0,0,0,22/07/2011
1,8005,Município,Paraíba do Sul,3303708,RJ,None,Arquivado por decurso de prazo,17944.000693/2014-97,2015-02-18T02:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,5800000.0,0,0,25/03/2015
2,6981,Município,Rio de Janeiro,3304557,RJ,None,Deferido,17944.000472/2007-90,2007-04-30T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,7403640.0,1,0,25/05/2007


In [7]:
df.dtypes #verica os tipos de dados

id_pleito                  int64
tipo_interessado          object
interessado               object
cod_ibge                   int64
uf                        object
num_pvl                   object
status                    object
num_processo              object
data_protocolo            object
tipo_operacao             object
finalidade                object
tipo_credor               object
credor                    object
moeda                     object
valor                    float64
pvl_assoc_divida           int64
pvl_contradado_credor      int64
data_status               object
dtype: object

In [8]:
# 2) Seu código aqui
# conferindo os status mais frequentes
df.status.value_counts()

status
Deferido                                                 106
Arquivado                                                 53
Regularizado                                              26
Arquivado por decurso de prazo                            24
Deferido (PVL-IF)                                         24
Encaminhado à PGFN com manifestação técnica favorável     14
Arquivado a pedido                                         8
Em análise                                                 2
Em retificação pelo interessado                            2
Arquivado a pedido (PVL-IF)                                1
Em retificação pelo credor                                 1
Arquivado pela STN                                         1
Name: count, dtype: int64

In [41]:
# os status mais frenquentes foram
# Deferido, Deferido (PVL-IF) e Arquivado
statusfreq = df.status.value_counts()
print('O status Deferido com uma frequencia de',statusfreq.iloc[0],'vezes.')
print('O status Arquivado com uma frequencia de',statusfreq.iloc[1],'vezes.')
print('O status Regularizado com uma frequencia de',statusfreq.iloc[2],'vezes.')

O status Deferido com uma frequencia de 106 vezes.
O status Arquivado com uma frequencia de 53 vezes.
O status Regularizado com uma frequencia de 26 vezes.


In [10]:
# 3) Seu código aqui
df['data_status'] = df['data_status'].astype(str)

In [11]:
df['ano_status'] = df['data_status'].str.split('/').str[2]

In [42]:
#4) Frequencia de cada ano
ano_freq = df.ano_status.value_counts() 
ano_freq

ano_status
2008    37
2007    29
2023    28
2014    20
2010    19
2019    16
2020    15
2013    15
2012    13
2022    11
2011    10
2009     9
2015     8
2021     5
2017     5
2018     4
2016     4
2002     4
2024     4
2006     4
2003     1
2004     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [17]:
#1) Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado ('Estado'ou Município),
#   e que devolve os dados da consulta no formato DataFrame.
def estado_de_interesse(UF,tipo_interessado):
    URL = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={UF}&tipo_interessado={tipo_interessado}'
    r = requests.get(URL)
    if r.status_code == 200:
        data_json = r.json()
        df = pd.DataFrame(data_json['items'])
        return df
    else:
        return('Erro ao fazer o request. Erro: {r.status_code}')

In [18]:
# 2) Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com status em 'Arquivado por decurso de prazo'
#    estão registradas?
UF = 'MG'
tipo_interessado = 'Estado'

In [19]:
df_MG = estado_de_interesse(UF,tipo_interessado)

In [20]:
df_MG.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13695,Estado,Minas Gerais,31,MG,None,Arquivado,19405.000053/2003-57,2003-08-25T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,35000000.0,0,0,20/02/2004
1,13587,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000031/2004-78,2004-11-27T02:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento rural,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,35000000.0,1,0,09/12/2004


In [21]:
status_mg = df_MG.status.value_counts()
status_mg 

status
Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado                                                 3
Arquivado pela STN                                        3
Indeferido                                                1
Em retificação pelo interessado                           1
Arquivado por decurso de prazo                            1
Name: count, dtype: int64

In [23]:
# Imprimindo apenas o status pedido
print('O status arquivado por decurso de prazo possui',status_mg.iloc[-1],'registros.')

O status arquivado por decurso de prazo possui 1 registros.


In [24]:
# 3) Qual é o município da Bahia com mais solicitações deferidas?
UF1 = 'BA'
tipo_interessado1 = 'Município'
df_BA = estado_de_interesse(UF1,tipo_interessado1)

In [33]:
df_BA.head(5)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,7069,Município,Mucugê,2921906,BA,None,Deferido,17944.000490/2010-77,2010-04-28T03:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,65000.0,0,0,20/05/2010
1,7806,Município,Baianópolis,2902500,BA,None,Deferido,17944.000658/2012-14,2012-05-03T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,900000.0,0,0,29/05/2012
2,11837,Município,Bonito,2904050,BA,None,Arquivado por decurso de prazo,17944.001539/2013-51,2013-11-01T02:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,300000.0,0,0,08/09/2014
3,11616,Município,Coribe,2909109,BA,None,Arquivado,17944.001489/2007-64,2008-01-07T02:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,500000.0,0,0,11/01/2008
4,7483,Município,Anguera,2901502,BA,None,Deferido,17944.000590/2010-01,2011-04-04T03:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,65000.0,0,0,19/04/2011


In [36]:
BA_mun_deferido = df_BA[df_BA['status']== 'Deferido'].interessado.value_counts()
BA_mun_deferido

interessado
Luís Eduardo Magalhães    16
Vitória da Conquista      12
Lauro de Freitas          11
Camaçari                  10
Barreiras                 10
                          ..
Salinas da Margarida       1
Xique-Xique                1
Andorinha                  1
Paulo Afonso               1
Cafarnaum                  1
Name: count, Length: 185, dtype: int64

In [40]:
print('O município com maior numero de solicitaçoes deferidas foi: Luís Eduardo Magalhães com',BA_mun_deferido.iloc[0],'solicitaçoes.')

O município com maior numero de solicitaçoes deferidas foi: Luís Eduardo Magalhães com 16 solicitaçoes.


In [29]:
# 4) Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

df_estado = estado_de_interesse('BA','Estado')

In [30]:
df_estado.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,8858,Estado,Bahia,29,BA,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000853/2014-06,2014-07-10T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,400000000.0,1,0,16/07/2014
1,12327,Estado,Bahia,29,BA,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001659/2008-91,2009-03-24T03:00:00Z,Operação contratual externa (com garantia da U...,Fortalecimento Institucional,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,409000000.0,1,0,27/03/2009


In [90]:
df_estado.to_csv('Solicitacoes estado da Bahia') #salvando o arquivo em csv